In [ ]:
# импорт зависимостей
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# настройка matplotlib
%config InlineBackend.figure_format = "retina"
plt.style.use("seaborn-v0_8-notebook")

# путь к данным
DATA_DIR = Path("../data")

##### **Гистограмма изображения**

In [ ]:
image = cv2.imread(
    DATA_DIR / "strawberries_coffee.png", 
    cv2.IMREAD_GRAYSCALE
)

H, W = image.shape

hist, bins = np.histogram(
    image.ravel(), 
    bins=256, 
    range=[0, 256]
)

prob = hist / (H * W)

# visualization
plt.figure(figsize=(10, 4))

plt.subplot(121)
plt.axis("off")
plt.imshow(image, vmin=0, vmax=255, cmap="gray")

plt.subplot(122)
plt.hist(bins[:-1], bins, weights=prob)

plt.tight_layout()

##### **Эквализация гистограммы цветного изображения**

In [ ]:
src_img = cv2.imread(
    DATA_DIR / "haze.png", 
    cv2.IMREAD_COLOR_RGB
)

Y, Cr, Cb = cv2.split(
    cv2.cvtColor(src_img, cv2.COLOR_RGB2YCrCb)
)

Y = cv2.equalizeHist(Y)

dst_img = cv2.cvtColor(
    cv2.merge([Y, Cr, Cb]),
    cv2.COLOR_YCrCb2RGB
)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 16))

ax1.imshow(src_img, vmin=0, vmax=255)
ax1.axis("off")

ax2.imshow(dst_img, vmin=0, vmax=255)
ax2.axis("off")

plt.tight_layout()


##### **CLAHE**

In [ ]:
src_img = cv2.imread(
    DATA_DIR / "einstein.png", 
    cv2.IMREAD_GRAYSCALE
)

clahe = cv2.createCLAHE(
    clipLimit=10, 
    tileGridSize=(4, 4)
)

dst_img = clahe.apply(src_img)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 16))

ax1.imshow(src_img, vmin=0, vmax=255, cmap="gray")
ax1.axis("off")

ax2.imshow(dst_img, vmin=0, vmax=255, cmap="gray")
ax2.axis("off")

plt.tight_layout()

##### **Вычитание изображений**
$$
    A - B = \left\{a_{ij}-b_{ij}\right\}
$$

In [ ]:
image1 = cv2.imread(DATA_DIR / "angiography1.tif")
image2 = cv2.imread(DATA_DIR / "angiography2.tif")

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 16))

ax1.imshow(image1, vmin=0, vmax=255, cmap="gray")
ax1.axis("off")

ax2.imshow(image2, vmin=0, vmax=255, cmap="gray")
ax2.axis("off")

plt.tight_layout()

In [ ]:
# numpy реализация вычитания
sub_image = np.clip(
    image1.astype(int) - image2.astype(int), 
    min=0, max=255
).astype(np.uint8)

# opencv реализация вычитания
sub_image = cv2.subtract(image1, image2)

# инвертация яркости
sub_image = 255 - sub_image

# нормализация изображения
sub_image = (sub_image - sub_image.min()) / (sub_image.max() - sub_image.min())
sub_image = (255 * sub_image).astype(np.uint8)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 16))

ax1.imshow(image2, vmin=0, vmax=255, cmap="gray")
ax1.axis("off")

ax2.imshow(sub_image, vmin=0, vmax=255, cmap="gray")
ax2.axis("off")

plt.tight_layout()

##### **Повышение резкости с помощью фильтра Лапласа**

$$
    g = f - \Delta f
$$
Девятиточечная разностная схема оператора Лапласа
$$
    \Delta f\approx
    \begin{bmatrix*}[r]
        1 & 1 & 1 \\
        1 & -8 & 1 \\
        1 & 1 & 1
    \end{bmatrix*}
    \circ f
$$

In [ ]:
src_image = cv2.imread(
    DATA_DIR / "blurry_moon.tif", 
    cv2.IMREAD_GRAYSCALE
)

w = np.array([
    [1, 1, 1],
    [1, -8, 1],
    [1, 1, 1]
])

laplace_image = cv2.filter2D(
    src=src_image, 
    ddepth=-1, 
    kernel=w
)

dst_image = cv2.subtract(src_image, laplace_image)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 16))

ax1.imshow(src_image, vmin=0, vmax=255, cmap="gray")
ax1.axis("off")

ax2.imshow(dst_image, vmin=0, vmax=255, cmap="gray")
ax2.axis("off")

plt.tight_layout()

##### **Уменьшение шума (noise reduction)**

In [ ]:
src_image = cv2.imread(
    DATA_DIR / "puppy.png", 
    cv2.IMREAD_GRAYSCALE
)

# медианный фильтр 15 x 15
median_image = cv2.medianBlur(
    src_image, 
    ksize=15
)

# гауссовская фильтрация 15 x 15
gauss_image = cv2.GaussianBlur(
    src_image, 
    ksize=(15, 15), 
    sigmaX=0
)

# билатеральная фильтрация sigma_c = 15, sigma_s = 15
bilat_image = cv2.bilateralFilter(
    src_image, 
    d=-1, 
    sigmaColor=15, 
    sigmaSpace=15
)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(
    2, 2, 
    figsize=(8, 8)
)

ax1.imshow(src_image, vmin=0, vmax=255, cmap="gray")
ax1.set_title("исходное изображение")
ax1.axis("off")

ax2.imshow(median_image, vmin=0, vmax=255, cmap="gray")
ax2.set_title("медианный фильтр")
ax2.axis("off")

ax3.imshow(gauss_image, vmin=0, vmax=255, cmap="gray")
ax3.set_title("гауссовский фильтр")
ax3.axis("off")

ax4.imshow(bilat_image, vmin=0, vmax=255, cmap="gray")
ax4.set_title("билатеральный фильтр")
ax4.axis("off")

plt.tight_layout()

#### **Домашняя работа 3**

##### **Задача**

- С помощью библиотеки `numpy` реализовать билатеральную фильтрацию 
одноканального изображения (размеры входного и выходного изображения должны совпадать).

    ```python
        def bilateral_filter(
            src_image, 
            sigma_space, 
            sigma_color
        ):
            # TODO
            assert src_image.shape == dst_image.shape
            return dst_image
    ```

- Вычисления сделать как можно эффективней: необходимо использовать векторные операции, распараллеливание (например c помощью numba кто умеет).

- Протестировать билатеральную фильтрацию на изображении `data/puppy.png`.

- Результаты OpenCV и вашей реализации билатеральной фильтрации сохранить в выходные изображения. Сравнить с OpenCV реализацией по визуальному качеству и времени выполнения.

- Пример параметров в файле `params.yaml`:

```yaml
    homework_03:
        input_image_path: data/puppy.png
        output_image_path: data/output.png
        opencv_output_image_path: data/cv_output.png
        sigma_space: 15
        sigma_color: 15
```

##### **Теория**

- гистограмма изображения
- эквализация гистограммы изображения
- алгоритм CLAHE
- повышение резкости изображения с помощью фильтра Лапласа
- повышение резкости изображения с помощью нерезкого маскирования
- модель шума "соль и перец", уменьшение шума с помощью медианной фильтрации
- модель аддитивного гауссовского шума, уменьшение шума с помощью фильтра Гаусса
- билатеральная фильтрация изображений